In [6]:
from IPython.display import Markdown, clear_output
import ipywidgets as widgets
from protomol.rd import mol
from protomol.util import render, update_db
from protomol.util.backend import Append_SQL, Query_SQL, Styles, write_orca


def on_query_smiles_btn(b):
    clear_output()
    display(header_box)
    if smiles_cbo.value not in smiles_cbo.options:
        display(write_smiles_box)
        render.from_smiles(smiles_cbo.value, add_labels=True)
    else:
        calcs = Query_SQL.calculations(smiles_cbo.value)
        if len(calcs) == 0:
            sh_cmd = write_orca(smiles_cbo.value, 2)
            display(Markdown(sh_cmd))
        else:
            # Update methods_cbo to only show methods for this SMILES
            available_methods = sorted(set(row[0] for row in calcs))


def on_write_smiles_btn(b):
    Append_SQL.smiles(smiles_cbo.value, multip_int.value)
    # Also call write_orca with the selected method
    sh_cmd = write_orca(
        smiles_cbo.value, new_methods_cbo.value, multiplicity=multip_int.value
    )
    clear_output()
    display(
        f"Added {smiles_cbo.value} with multiplicity {multip_int.value} to data.db. Please restart."
    )
    display(Markdown(sh_cmd))


def on_refresh_db_btn(b):
    update_db.refresh()


def on_visualize_xyz_btn(b):
    clear_output()
    display(header_box, calcs_box)
    rows = Query_SQL.calculations(smiles_cbo.value, methods_cbo.value)
    if len(rows) == 1:
        alpha_indices, beta_indices = mol.beta_cleavage_indices(smiles_cbo.value)
        alpha_cbo.options = alpha_indices
        beta_cbo.options = beta_indices
        radical_indices, proton_indices = mol.radical_indices(smiles_cbo.value)
        proton_cbo.options = proton_indices
        radical_cbo.options = radical_indices
        display(new_calc_box)
        render.from_xyz_block(Query_SQL.xyzs(rows[0][0]), add_labels=True)
    elif len(rows) > 1:
        raise NotImplementedError
    else:
        display("No matching XYZ in database.")


def on_visualize_traj_btn(b):
    clear_output()
    display(header_box, calcs_box)
    rows = Query_SQL.calculations(smiles_cbo.value, methods_cbo.value)
    if len(rows) == 1:
        render.from_allxyz_block(Query_SQL.traj(rows[0][0]))
    else:
        display("No matching traj in database.")


# Unified submit button logic
def on_submit_btn(b):
    # Check which indices are selected to determine the mechanism
    # Beta cleavage: both alpha and beta indices are selected (not None)
    # Proton transfer: both proton and radical indices are selected (not None)
    # Otherwise: optimization
    rows_opt = Query_SQL.calculations(smiles_cbo.value, methods_cbo.value)
    # Beta cleavage
    if alpha_cbo.value is not None and beta_cbo.value is not None:
        rows = Query_SQL.calculations(smiles_cbo.value, methods_cbo.value, alpha_cbo.value, beta_cbo.value)
        if len(rows) == 0:
            sh_cmd = write_orca(
                smiles_cbo.value,
                new_methods_cbo.value,
                idx1=alpha_cbo.value,
                idx2=beta_cbo.value,
                mechanism="beta cleavage",
            )
            display(Markdown(sh_cmd))
        else:
            display("Issue with database.")
    # Proton transfer
    elif proton_cbo.value is not None and radical_cbo.value is not None:
        rows = Query_SQL.calculations(smiles_cbo.value, methods_cbo.value, proton_cbo.value, radical_cbo.value)
        if len(rows) == 0:
            sh_cmd = write_orca(
                smiles_cbo.value,
                new_methods_cbo.value,
                idx1=proton_cbo.value,
                idx2=radical_cbo.value,
                mechanism="proton transfer",
            )
            display(Markdown(sh_cmd))
        else:
            display("Issue with database.")
    # Optimization (default)
    elif len(rows_opt) == 1:
        sh_cmd = write_orca(smiles_cbo.value, new_methods_cbo.value, rows_opt[0][0])
        display(Markdown(sh_cmd))
    else:
        display("No matching XYZ in database.")


# Header box
smiles_cbo = widgets.Combobox(
    options=Query_SQL.smiles(), layout=Styles.scale_width("50%")
)
query_smiles_btn = widgets.Button(
    description="Submit Query", layout=Styles.scale_width("20%")
)
query_smiles_btn.on_click(on_query_smiles_btn)
refresh_db_btn = widgets.Button(
    description="Refresh Database", layout=Styles.scale_width("20%")
)
refresh_db_btn.on_click(on_refresh_db_btn)
header_box = widgets.HBox([smiles_cbo, query_smiles_btn, refresh_db_btn])

# New SMILES box
multip_int = widgets.IntText(
    description="Multiplicity: ", layout=Styles.scale_width("20%")
)
write_smiles_btn = widgets.Button(
    description="Write SMILES to database", layout=Styles.scale_width("20%")
)
write_smiles_btn.on_click(on_write_smiles_btn)
write_smiles_box = widgets.HBox([multip_int, write_smiles_btn])

# Methods Box
methods_cbo = widgets.Dropdown(options=[], layout=Styles.scale_width("50%"))
visualize_xyz_btn = widgets.Button(
    description="Visualize XYZ", layout=Styles.scale_width("20%")
)
visualize_xyz_btn.on_click(on_visualize_xyz_btn)
visualize_traj_btn = widgets.Button(
    description="Visualize Traj", layout=Styles.scale_width("20%")
)
visualize_traj_btn.on_click(on_visualize_traj_btn)
calcs_box = widgets.HBox([methods_cbo, visualize_xyz_btn, visualize_traj_btn])

# New Calculation Box
new_methods_cbo = widgets.Dropdown(
    options=Query_SQL.methods(),
    description="New Method",
    layout=Styles.scale_width("40%"),
)
# Unified submit button
submit_btn = widgets.Button(
    description="Submit Calculation", layout=Styles.scale_width("15%")
)
submit_btn.on_click(on_submit_btn)
new_opt_box = widgets.HBox([new_methods_cbo, submit_btn])

beta_cleavage_lbl = widgets.Label(
    value="Beta Cleavage: ", layout=Styles.scale_width("10%")
)
alpha_cbo = widgets.Dropdown(
    options=[], description="α", layout=Styles.scale_width("15%")
)
beta_cbo = widgets.Dropdown(
    options=[], description="β", layout=Styles.scale_width("15%")
)
# Remove old submit_beta_btn
beta_cleavage_box = widgets.HBox(
    [beta_cleavage_lbl, alpha_cbo, beta_cbo],
)

proton_transfer_lbl = widgets.Label(
    value="Proton Transfer: ", layout=Styles.scale_width("10%")
)
proton_cbo = widgets.Dropdown(
    options=[], description="Proton", layout=Styles.scale_width("15%")
)
radical_cbo = widgets.Dropdown(
    options=[], description="Radical", layout=Styles.scale_width("15%")
)
# Remove old submit_transfer_btn
proton_transfer_box = widgets.HBox(
    [proton_transfer_lbl, proton_cbo, radical_cbo],
)

path_idx_int = widgets.IntText(description="Index: ", layout=Styles.scale_width("30%"))

new_calc_box = widgets.VBox([new_opt_box, beta_cleavage_box, proton_transfer_box])

display(header_box)

In [2]:
functional = "XTB"
basis = "N/A"
method = "OPT SCAN"
inp_template = (
    "%PAL NPROCS 8 END\n"
    "%maxcore 750\n"
    "! XTB OPT\n"
    "% geom\n"
    "   scan\n"
    "       B [idx1] [idx2] = [start], [end], 18\n"
    "   end\n"
    "end\n"
    "* xyzfile 0 [multiplicity] init.xyz\n"
)
submit_template = (
    "#!/bin/bash\n"
    "#SBATCH --partition=batch\n"
    "#SBATCH --gres=lscratch:10\n"
    "#SBATCH --job-name=[smiles]_XTB\n"
    "#SBATCH --nodes=1\n"
    "#SBATCH --ntasks=8\n"
    "#SBATCH --ntasks-per-node=8\n"
    "#SBATCH --cpus-per-task=1\n"
    "#SBATCH --time=00:30:00\n"
    "#SBATCH --mem-per-cpu=1G\n"
    "cd ${SLURM_SUBMIT_DIR}\n"
    "mkdir -p /lscratch/${USER}/${SLURM_JOB_ID}\n"
    'cp -r "${SLURM_SUBMIT_DIR}/." /lscratch/${USER}/${SLURM_JOB_ID}\n'
    "cd /lscratch/${USER}/${SLURM_JOB_ID}\n\n"
    "module load ORCA/6.1\n"
    "$(which orca) CALC.inp > CALC.log\n"
    "cp -r /lscratch/${USER}/${SLURM_JOB_ID} ${SLURM_SUBMIT_DIR}\n"
    "rm -rf /lscratch/${USER}/${SLURM_JOB_ID}\n"
)

Append_SQL._method(functional, basis, method, inp_template, submit_template)

In [33]:
import ipywidgets as widgets
from IPython.display import clear_output
options = []
stacks = []

# New Optimization
def on_update_smiles_btn(b):
    clear_output()
    display(controller)
    display("Test")

options.append("New Optimization")
smiles_cbo = widgets.Combobox(
    options=Query_SQL.smiles(), layout=Styles.scale_width("50%")
)
smiles_cbo.observe(on_update_smiles_btn)
update_smiles_btn = widgets.Button(value="Update SMILES", layout=Styles.scale_width("25%"))
stacks.append([smiles_cbo, update_smiles_btn])

# Visualize XYZ
options.append("Visualize XYZ")
slider = widgets.IntSlider()
stacks.append(slider)

# Main controller
def on_refresh_db_btn(b):
    update_db.refresh()
    display("Submitted database update.")

dropdown = widgets.Dropdown(options=options, layout=Styles.scale_width("50%"))
refresh_db_btn = widgets.Button(
    description="Refresh Database", layout=Styles.scale_width("25%")
)
refresh_db_btn.on_click(on_refresh_db_btn)
header_box = widgets.HBox([dropdown, refresh_db_btn])
stack = widgets.Stack(stacks, selected_index=0, layout=Styles.scale_width("100%"))
widgets.jslink((dropdown, 'index'), (stack, 'selected_index'))
controller = widgets.VBox([header_box, stack])
display(controller)

TraitError: The 'children' trait of a Stack instance contains an Instance of a TypedTuple which expected a Widget, not the list [Combobox(value='', layout=Layout(height='30', width='50%'), options=('C=C', 'CC[CH]C1CO1', 'C[CH]CC1CO1', 'O=O', '[CH2]CCC1CO1')), Button(layout=Layout(height='30', width='25%'), style=ButtonStyle())].